<a href="https://colab.research.google.com/github/diogopaz/projeto-marvel/blob/code-documentation/projeto_marvel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importando bibliotecas necessárias

In [ ]:
!pip install dotenv --quiet
!pip install plotly --quiet

In [ ]:
import requests
import hashlib
import time
import os
import pandas as pd
from dotenv import load_dotenv
import sqlite3
from google.colab import userdata
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

## Carregando variáveis de ambiente
Estamos fazendo upload do arquivo `.env` com as chaves de acesso à API no ambeinte de execução do colab. Como alternativa, também é possível salvar as chaves nos `secrets` do colab e usá-las nas requisições.

In [ ]:
load_dotenv(dotenv_path='/content/.env')
public_key = userdata.get('MARVEL_PUBLIC_KEY') or os.getenv('MARVEL_PUBLIC_KEY')
private_key = userdata.get('MARVEL_PRIVATE_KEY') or os.getenv('MARVEL_PRIVATE_KEY')

## Montando os parâmetros para realizar a requisição

In [ ]:
def get_auth_params():
    ts = str(time.time())
    to_hash = ts + private_key + public_key
    hash_md5 = hashlib.md5(to_hash.encode()).hexdigest()
    return {
        'ts': ts,
        'apikey': public_key,
        'hash': hash_md5
    }

## Buscando os dados na API da Marvel

A função `fetch_from_marvel` retorna todos os dados do endpoint passado como parâmetro em um DataFrame e exporta os dados brutos para um arquivo `.csv`.

In [ ]:
def fetch_from_marvel(endpoint, limit = 100, offset = 0):
  "Busca todos os resultados de um endpoint paginado da Marvel API"
  limit = limit
  offset = offset
  all_results = []
  total = 1
  req_count = 0

  print(f"Iniciando requisições para o endpoint: {endpoint}")

  while offset < total:
    params = get_auth_params()
    params["limit"] = limit
    params["offset"] = offset

    try:
      response = requests.get(endpoint, params=params)
      response.raise_for_status()
      data = response.json().get("data", {})
      total = data['total']
      results = data.get("results", [])
      if not results:
        break

      all_results.extend(results)
      print(f"Recebidos {len(results)} resultados (offset {offset})")

      offset += limit
      req_count += 1
      time.sleep(1)
    except Exception as e:
      print(f"Erro em offset {offset}: {e}. Pulando...")
      offset += limit
      req_count += 1
      time.sleep(1)

  # Exportando dados brutos para csv
  df = pd.DataFrame(all_results)
  df.to_csv(f'{endpoint.split("/")[-1]}.csv')

  print(f'''
  Sucesso! Recebidos {len(all_results)} resultados.
  Requisições realizadas: {req_count}.
  Dados brutos salvos em: {endpoint.split("/")[-1]}.csv
  ''')

  return df

## Filtrando os dados e salvando no banco

### Funções auxiliares

A função `to_db` salva o DataFrame com dados filtrados no banco de dados selecionado.

In [ ]:
def to_db(df, table_name, db_name):
  try:
    conn = sqlite3.connect(f"{db_name}.db")
    df.to_sql(table_name, conn, index=False, if_exists='replace')
    conn.commit()
    conn.close()
    print('Dados salvos no banco')
  except Exception as e:
    conn.close()
    print(e)

A função `extract_comic_prices` cria um DataFrame relacionando cada quadrinho com seus preços.

In [ ]:
# Cria um DataFrame relacionando cada quadrinho com seus preços
def extract_comic_prices(df_comics):
  records = []
  for _, row in df_comics.iterrows():
    comic_id = row.get('id')
    for price in row.get('prices', []):
      records.append({
        'comic_id': comic_id,
        'type': price.get('type', ''),
        'price': price.get('price', 0.0)
      })
  return pd.DataFrame(records)

A função `extract_comic_creators` cria um Dataframe que relaciona cada quadrinho com seus criadores

In [ ]:
# Cria um DataFrame relacionando cada quadrinho com seus criadores
def extract_comic_creators(df_comics):
  records = []
  for _, row in df_comics.iterrows():
    comic_id = row.get('id')
    for creator in row.get('creators', {}).get('items', []):
      try:
        resource_uri = creator.get('resourceURI', '')
        creator_id = int(resource_uri.strip().split('/')[-1])
        role = creator.get('role', '')
        records.append({
          'comic_id': comic_id,
          'creator_id': creator_id,
          'role': role
        })
      except (IndexError, ValueError):
        continue
  return pd.DataFrame(records)

A função `extract_character_comics` cria um DataFrame com a relação entre os personagens (passados como uma lista de id's) e os quadrinhos que eles possuem.

In [ ]:
def extract_character_comics(character_ids):
    all_pairs = []

    for character_id in character_ids:
        print(f"Buscando comics para o personagem {character_id}...")
        try:
            endpoint = f"https://gateway.marvel.com/v1/public/characters/{character_id}/comics"
            limit = 100
            offset = 0
            total = 1

            while offset < total:
                params = get_auth_params()
                params.update({"limit": limit, "offset": offset})
                response = requests.get(endpoint, params=params)
                response.raise_for_status()
                data = response.json().get("data", {})
                total = data.get("total", 0)
                results = data.get("results", [])

                for comic in results:
                    comic_id = comic.get("id")
                    if comic_id:
                        all_pairs.append({
                            "character_id": character_id,
                            "comic_id": comic_id
                        })

                offset += limit
                time.sleep(0.1)

        except Exception as e:
            print(f"Erro ao processar o personagem {character_id}: {e}")

    df = pd.DataFrame(all_pairs)
    return df


### Tratamento dos dados
A partir dos resultados retornados de cada endpoint e tranformados em DataFrames, tratamos os dados para salvar no banco apenas as informações relevantes.

Personagens:

In [ ]:
characters = fetch_from_marvel('https://gateway.marvel.com/v1/public/characters')
characters['comics_available'] = characters['comics'].apply(lambda x: x.get('available'))
characters = characters[['id', 'name', 'description', 'modified', 'comics_available']]

# Para nossa análise, precisamos armazenar a relação personagem/quadrinho apenas dos 10 personagens que mais possuem quadrinhos
top_10_ids = characters.sort_values(by="comics_available", ascending=False).head(10)["id"].tolist()
character_comics = extract_character_comics(top_10_ids)

# Salvando o personagens e a relação personagem/quadrinho no banco
to_db(characters, table_name='characters', db_name='teste4')
to_db(character_comics, table_name='character_comics', db_name='teste4')

Quadrinhos:

In [ ]:
comics = fetch_from_marvel('https://gateway.marvel.com/v1/public/comics')
comic_prices = extract_comic_prices(comics)
comic_creators = extract_comic_creators(comics)

comics['variant_count'] = comics['variants'].apply(lambda x: len(x))
comics['page_count'] = comics['pageCount']
comics = comics[['id', 'title', 'page_count', 'variant_count']]

# Salvando os dados de quadrinhos e as relações quadrinho/preços e quadrinho/criadores no banco:
to_db(comics, table_name='comics', db_name='marvel')
to_db(comic_prices, table_name='comic_prices', db_name='marvel')
to_db(comic_creators, table_name='comic_creators', db_name='marvel')

Criadores:

In [ ]:
creators = fetch_from_marvel('https://gateway.marvel.com/v1/public/creators', 20)
# Pegando 20 resultados por requisição, pois este endpoint possui muitos offsets que retornam erro

creators = creators[['id', 'firstName', 'middleName', 'lastName', 'suffix', 'fullName', 'modified']]
to_db(creators, table_name='creators', db_name='marvel')

## Insights

In [ ]:
conn = sqlite3.connect('marvel.db')
cursor = conn.cursor()
cursor.execute('select * from characters')
charr = cursor.fetchall()
df_char = pd.DataFrame(charr, columns=['id', 'name', 'description', 'modified', 'comic_available'])
df_char.sort_values('comic_available', ascending=False)

In [ ]:
df_char['comic_available'] = pd.to_numeric(df_char['comic_available'], errors='coerce')
df_counts = df_char['comic_available'].value_counts().sort_index()

plt.figure(figsize=(10, 6))
plt.scatter(df_counts.index, df_counts.values, color='salmon', edgecolor='black')
plt.title('Distribuição de Participações em Quadrinhos por Personagem')
plt.xlabel('Número de Quadrinhos (comic_available)')
plt.ylabel('Número de Personagens')

step = 500
x_max = df_counts.index.max()
plt.xticks(range(0, int(x_max)+1, step))

plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

In [ ]:
df_char['comic_available'] = pd.to_numeric(df_char['comic_available'], errors='coerce')
df_counts = df_char['comic_available'].value_counts().sort_index()
df_plot = df_counts.reset_index()
df_plot.columns = ['comic_available', 'num_personagens']

fig = px.scatter(
    df_plot,
    x='comic_available',
    y='num_personagens',
    labels={
        'comic_available': 'Número de Quadrinhos',
        'num_personagens': 'Número de Personagens'
    },
    title='Distribuição de Participações em Quadrinhos por Personagem',
    hover_data={
        'comic_available': True,
        'num_personagens': True
    }
)

fig.update_traces(marker=dict(color='salmon', line=dict(width=1, color='black')))
fig.update_layout(width=900, height=500)
fig.show()

In [ ]:
df_char['comic_available'] = pd.to_numeric(df_char['comic_available'], errors='coerce')
df_char['modified'] = pd.to_datetime(df_char['modified'], errors='coerce')

df_plot = df_char.dropna(subset=['comic_available', 'modified', 'name'])

fig = px.scatter(
    df_plot,
    x='comic_available',
    y='modified',
    hover_name='name',
    labels={
        'comic_available': 'Número de Quadrinhos',
        'modified': 'Última Modificação'
    },
    title='Participações em Quadrinhos vs. Última Modificação (por Personagem)'
)

fig.update_traces(marker=dict(color='salmon', size=7, line=dict(width=1, color='black')))
fig.update_layout(width=950, height=550)
fig.show()

In [ ]:
df_char['comic_available'] = pd.to_numeric(df_char['comic_available'], errors='coerce')
df_char['modified'] = pd.to_datetime(df_char['modified'], errors='coerce')
df_plot = df_char.dropna(subset=['comic_available', 'modified', 'name']).copy()

bins = [0, 10, 50, 100, 500, 1000, 2000, 3000, 4000,float('inf')]
labels = ['0–10', '11–50', '51–100', '101–500', '501-1000','1001-2000','2001-3000','3001-4000','4000+']
df_plot['faixa_comics'] = pd.cut(df_plot['comic_available'], bins=bins, labels=labels, include_lowest=True)

df_plot['ano_mod'] = df_plot['modified'].dt.year.astype(str)

fig = px.scatter(
    df_plot,
    x='comic_available',
    y='modified',
    color='faixa_comics',
    hover_name='name',
    labels={
        'comic_available': 'Número de Quadrinhos',
        'modified': 'Data de Modificação',
        'faixa_comics': 'Faixa de Participações'
    },
    title='Participações em Quadrinhos vs. Última Modificação (por Personagem)',
    category_orders={"faixa_comics": labels}
)
fig.update_traces(marker=dict(size=7, line=dict(width=1, color='black')))
fig.update_layout(width=1000, height=600)

anos_unicos = sorted(df_plot['ano_mod'].unique())

fig.update_layout(
    updatemenus=[
        {
            "buttons": [
                {"label": "Todos os anos", "method": "restyle", "args": ["visible", [True]*len(df_plot)]}
            ] + [
                {
                    "label": ano,
                    "method": "update",
                    "args": [
                        {"visible": df_plot['ano_mod'] == ano},
                        {"title": f"Personagens modificados em {ano}"}
                    ]
                }
                for ano in anos_unicos
            ],
            "direction": "down",
            "showactive": True,
            "x": 1.1,
            "xanchor": "left",
            "y": 1.15,
            "yanchor": "top"
        }
    ]
)

fig.show()


In [ ]:
df_char['comic_available'] = pd.to_numeric(df_char['comic_available'], errors='coerce')
df_char['modified'] = pd.to_datetime(df_char['modified'], errors='coerce')
df_plot = df_char.dropna(subset=['comic_available', 'modified', 'name']).copy()

# As cores estão aqui!!!!
bins = [0, 10, 50, 100, 500, 1000, 2000, 3000, 4000,float('inf')]
labels = ['0–10', '11–50', '51–100', '101–500', '501-1000','1001-2000','2001-3000','3001-4000','4000+']
cores = {
    '0–10': 'lightgray',
    '11–50': 'lightskyblue',
    '51–100': 'dodgerblue',
    '101–500': 'seagreen',
    '501-1000': 'crimson',
    '1001-2000':'indigo',
    '2001-3000':'gold',
    '3001-4000':'darkturquoise',
    '4000+':'darkblue'
}
df_plot['faixa_comics'] = pd.cut(df_plot['comic_available'], bins=bins, labels=labels, include_lowest=True)
df_plot['cor_faixa'] = df_plot['faixa_comics'].map(cores)


df_plot['ano_mod'] = df_plot['modified'].dt.year
anos_unicos = sorted(df_plot['ano_mod'].dropna().unique())


fig = go.Figure()

for ano in anos_unicos:
    df_ano = df_plot[df_plot['ano_mod'] == ano]
    fig.add_trace(go.Scatter(
        x=df_ano['comic_available'],
        y=df_ano['modified'],
        mode='markers',
        marker=dict(size=7, color=df_ano['cor_faixa'], line=dict(width=1, color='black')),
        name=str(ano),
        text=df_ano['name'],
        hovertemplate='<b>%{text}</b><br>Quadrinhos: %{x}<br>Modificado: %{y|%Y-%m-%d}<extra></extra>',
        visible=True if ano == anos_unicos[0] else False
    ))

buttons = [
    dict(
        label="Todos os anos",
        method="update",
        args=[{"visible": [True] * len(anos_unicos)},
              {"title": "Todos os Anos"}]
    )
]

for i, ano in enumerate(anos_unicos):
    vis = [False] * len(anos_unicos)
    vis[i] = True
    buttons.append(
        dict(
            label=str(ano),
            #color=cores,
            method="update",
            args=[{"visible": vis},
                  {"title": f"Personagens modificados em {ano}"}]
        )
    )

fig.update_layout(
    title="Participações em Quadrinhos vs. Data de Modificação (com Filtro por Ano)",
    xaxis_title="Número de Quadrinhos",
    yaxis_title="Data de Modificação",
    width=1000,
    height=600,
    updatemenus=[dict(
        type="dropdown",
        direction="down",
        buttons=buttons,
        x=1.1,
        xanchor="left",
        y=1.15,
        yanchor="top"
    )],
    legend_title_text="Faixas de Quadrinhos"
)
fig.show()
